In [1]:
import pandas as pd
import numpy as np
from mca import diagsvd
# import mca
import functools
import sklearn.model_selection as ms
from sklearn.preprocessing import StandardScaler as stdc
import gc
import _pickle as pickle

categorical = ['source_system_tab','source_type','gender','city','genre_ids','language']
numerical = ['song_length', 'song_year','bd']

# Partitioning

In [2]:
file = 'def_training.csv'
#categorical = ['source_system_tab','source_type','gender','city','genre_ids','language']
X = pd.read_csv('../Data/' + file,nrows=None, dtype={'target': np.uint8,'city':'category',
                                                     'gender':'category','source_system_tab':'category',
                                                     'source_type':'category','song_year':np.uint64,
                                                     'song_length':np.uint64,
                                                     'language':'category','genre_ids':'category'})

y = X['target']
X = X.drop(columns=['song_id','source_screen_name','registered_via',
                    'registration_init_time','expiration_date','age_range', 
                    'artist_name','composer','lyricist','name','registrant_code','country_code', 'Unnamed: 0' ])
    
X_num = X[numerical]
X_num.to_csv('X_num.csv')

del X_num
gc.collect()

print(X.dtypes)

X.head()

source_system_tab    category
source_type          category
target                  uint8
city                 category
bd                      int64
gender               category
song_length            uint64
genre_ids            category
language             category
song_year              uint64
dtype: object


,source_system_tab,source_type,target,city,bd,gender,song_length,genre_ids,language,song_year
0,explore,online-playlist,1,1,0,1,206471,359,52.0,2016
1,my library,local-playlist,1,13,24,0,284584,1259,52.0,1999
2,my library,local-playlist,1,13,24,0,225396,1259,52.0,2006
3,my library,local-playlist,1,13,24,0,255512,1019,-1.0,2010
4,explore,online-playlist,1,1,0,1,187802,1011,52.0,2016


In [3]:
# for c in categorical:
#     print(pd.crosstab(index=X["target"],columns=X[c]))


Particiono

In [4]:
DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')

print(DummiesX.shape)
DummiesX.head()

(7347633, 624)


,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_system_tab|settings,source_type|album,source_type|artist,...,language|-1.0,language|10.0,language|17.0,language|24.0,language|3.0,language|31.0,language|45.0,language|52.0,language|59.0,language|38.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
del X
gc.collect()

75

Elimino las columnas que tienen cero para todas las filas de algun subconjunto

In [6]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)


In [7]:
proportion = 0.4
(X_train, X_test, y_train, y_test) = ms.train_test_split(DummiesX, y, test_size=proportion, random_state=42, stratify=y)

X_train_sum = (X_train.values.sum(axis=0) != 0)
X_test_sum = (X_test.values.sum(axis=0) != 0)
index = np.logical_and(X_train_sum, X_test_sum)
X_train = X_train.loc[:, index]
X_test = X_test.loc[:, index]
idx = X_train.index

X_test['target'] = y_test
X_test.to_pickle('X_test.pkl')
del DummiesX, X_test, X_train_sum
gc.collect()
                      

10

# MCA

In [8]:
size = [X_train.shape[0], X_train.shape[1]]
N_all = np.sum(X_train.values[0:size[0],:])

Z_residual = X_train.values[0:size[0],:] / N_all
X_train.to_pickle('X_train.pkl')
del X_train
gc.collect()
Z_residual.shape

(4408579, 519)

In [9]:
Sum_r = np.sum(Z_residual, axis=1)
Sum_c = np.sum(Z_residual, axis=0)
print(size, Sum_r.shape, Sum_c.shape, N_all)

[4408579, 519] (4408579,) (519,) 26451364


In [10]:
Z_residual -= np.outer(Sum_r, Sum_c)


In [11]:
D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
del Sum_r
gc.collect()
D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
del Sum_c
gc.collect()

MemoryError: 

In [ ]:
MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)

del Z_residual
gc.collect()

In [ ]:
P, S, Q = np.linalg.svd(MCA_mat)

In [ ]:
Lam = S ** 2
Expl_var = Lam / np.sum(Lam)

# print('Eigen values are ', Lam)
# print('Explained variance of eigen vectors are ', Expl_var)

data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var)}
columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'
np.round(table2.astype(float), 4)
table2.to_csv('table2.csv')
del Lam, Expl_var
gc.collect()

In [ ]:
columns_keep = len(table2['τI'])
while(table2['τI'][0:columns_keep].sum()>0.9):
    columns_keep = columns_keep - 2
    

print('Varianza explicada tomando '+str(columns_keep)+' de '+ str(len(table2['τI']))+' features: ', table2['τI'][0:columns_keep].sum())

In [ ]:
S_d = diagsvd(S, size[0], size[1])


In [ ]:
F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows

In [ ]:
del  Q, P, MCA_mat, S_d, D_r_sqrt_mi, D_c_sqrt_mi
gc.collect()

In [ ]:

X_cat_trainMatrix= Matrix_mult(pd.read_pickle('X_train.pkl').values, G[:, :columns_keep]) / S[:columns_keep] / 10


In [ ]:
names = ['V' + str(i) for i in range(columns_keep)]
X_cat_train = pd.DataFrame(X_cat_trainMatrix, columns=names, index = idx)

del X_cat_trainMatrix
gc.collect()

X_num = pd.read_csv('X_num.csv',nrows=None, dtype={'song_year':np.uint64,
                                                     'song_length':np.uint64,
                                                     })

X_num = pd.DataFrame(stdc().fit_transform(X_num.values), columns=X_num.columns, index = X_num.index)
X_num = X_num.drop(columns=['Unnamed: 0'])
(X_train_num, X_test_num, y_train, y_test) = ms.train_test_split(X_num, y, test_size=proportion, random_state=42, stratify=y)

concat_Train = pd.concat([X_cat_train, X_train_num], axis=1, ignore_index=False)
concat_Train['target'] = y_train
concat_Train.to_csv('../Data/preprocessedTrainMCA.csv')

del X_cat_train,X_train_num
gc.collect()

In [ ]:
concat_Train.head()


In [ ]:
X_cat_test= pd.DataFrame(Matrix_mult(pd.read_pickle('X_test.pkl').drop(columns='target')
.values, G[:, :columns_keep]) / S[:columns_keep] / 10, columns=names, index =pd.read_pickle('X_test.pkl').index)

concat_Test = pd.concat([X_cat_test, X_test_num], axis=1, ignore_index=False)
concat_Test['target'] = y_test
concat_Test.to_csv('../Data/preprocessedTestMCA.csv')
concat_Test.isnull().sum().sum()

# Saving data